In [ ]:
# 我們的程式是參考其他人的程式寫出來的，要記得把這些資訊寫出來喔

# original code:
# Snake - v1.0
# License CC BY 4.0
# Thomas Le Menestrel - Colin Laganier - 2019.09.18

In [ ]:
# 使用函式庫
from random import randint
import pygame
import time

In [ ]:
# 遊戲進行所需的變數
playing = True               # 遊戲是否正在運行
snake_positions = []         # 蛇的身體位置
fruit_position = []          # 蘋果的位置
growth_position = [0, 0]     # 吃到蘋果時要長出來的那節身體所在的位子
speed = 100                  # 遊戲進行的速度，越大越慢
pause = True                # 遊戲是否暫停

move_up = False              # 蛇在往「上」走
move_down = False            # 蛇在往「下」走
move_left = False            # 蛇在往「左」走
move_right = True            # 蛇在往「右」走

# 常數。跟遊戲畫面大小有關，不用理會
BOARD_LEN = 800
GRID_CNT = 20
GRID_STEP = BOARD_LEN / GRID_CNT

# 顏色們。不用理會
GRID_CLRS = [(212, 191, 136), (230, 222, 185)]
BORDER_CLRS = [(58, 73, 40), (75, 107, 60)]

In [ ]:
# 函式
def project(pos):
    return [pos[0] * GRID_STEP, pos[1] * GRID_STEP]

In [ ]:
# 建立視窗
pygame.init()
window = pygame.display.set_mode((BOARD_LEN, BOARD_LEN))
windowRect = window.get_rect()
pygame.display.set_caption("Snake")

# 渲染視窗底色
cover = pygame.Surface(window.get_size())
cover = cover.convert()
cover.fill((250, 250, 250))
window.blit(cover, (0, 0))
for x in range(GRID_CNT):
    for y in range(GRID_CNT):
        rect = pygame.Rect(*project([x, y]), GRID_STEP, GRID_STEP)
        clr = GRID_CLRS[(x+y) % 2]
        if x == 0 or y == 0 or x == GRID_CNT - 1 or y == GRID_CNT - 1:
            clr = BORDER_CLRS[(x+y) % 2]
        pygame.draw.rect(cover, clr, rect)

window.blit(cover, (0, 0))
pygame.display.flip()

In [ ]:
# 載入圖片
head = pygame.image.load("./images/worm_head.png").convert_alpha() # 頭的圖片
head = pygame.transform.scale(head, (GRID_STEP * 1.3, GRID_STEP * 1.3))

body = pygame.image.load("./images/body.png").convert_alpha() # 身體的圖片
body = pygame.transform.scale(body, (GRID_STEP * 1.1, GRID_STEP * 1.1))

fruit = pygame.image.load("./images/fruit.png").convert_alpha() # 水果的圖片
fruit = pygame.transform.scale(fruit, (GRID_STEP * 1.2, GRID_STEP * 1.2))

In [ ]:
# 蛇的頭跟第一節身體的坐標
snake_positions.append([2, 1])
snake_positions.append([1, 1])

# 隨機產生水果的坐標
fruit_position = [randint(1, GRID_CNT - 2), randint(1, GRID_CNT - 2)]

In [ ]:
while playing == True:

    # 逐一檢查各種事件
    for event in pygame.event.get():

        # 離開遊戲（點擊右上角叉叉）
        if event.type == pygame.QUIT:
            playing = False

        # 按鍵盤
        if event.type == pygame.KEYDOWN:

            # 按空白鍵 => 暫停遊戲
            if event.key == pygame.K_SPACE:
                if pause == True:
                    pause = False
                elif pause == False:
                    pause = True

            # 按往上鍵 => 蛇要往上走
            elif event.key == pygame.K_UP:
                if move_down == False:
                    move_up = True
                    move_down = False
                    move_left = False
                    move_right = False

            # 按往下鍵 => 蛇要往下走
            elif event.key == pygame.K_DOWN:
                if move_up == False:
                    move_up = False
                    move_down = True
                    move_left = False
                    move_right = False
                    
            # 按往左鍵 => 蛇要往左走
            elif event.key == pygame.K_LEFT:
                if move_right == False:
                    move_up = False
                    move_down = False
                    move_left = True
                    move_right = False

            # 按往右鍵 => 蛇要往右走
            elif event.key == pygame.K_RIGHT:
                if move_left == False:
                    move_up = False
                    move_down = False
                    move_left = False
                    move_right = True

    if pause == False:
        # 身體往前進
        growth_position = snake_positions[-1]
        for i in range(len(snake_positions) - 1, 0, -1):
            snake_positions[i] = snake_positions[i - 1][:]

        # 頭往前進
        if move_up:
            snake_positions[0][1] = snake_positions[0][1] - 1
        elif move_down:
            snake_positions[0][1] = snake_positions[0][1] + 1
        elif move_left:
            snake_positions[0][0] = snake_positions[0][0] - 1
        elif move_right:
            snake_positions[0][0] = snake_positions[0][0] + 1

    # 撞到自己
    for i in range(1, len(snake_positions)):
        if snake_positions[0] == snake_positions[i]:
            playing = False

    # 撞到牆
    if snake_positions[0][0] >= GRID_CNT - 1:
        playing = False
    if snake_positions[0][0] <= 0:
        playing = False
    if snake_positions[0][1] >= GRID_CNT - 1:
        playing = False
    if snake_positions[0][1] <= 0:
        playing = False

    # 吃到蘋果
    if snake_positions[0] == fruit_position:

        # 產生蘋果
        fruit_in_body = True
        while fruit_in_body == True:
            fruit_position = [randint(1, GRID_CNT - 2), randint(1, GRID_CNT - 2)]
            fruit_in_body = False
            for i in range(len(snake_positions)):
                if snake_positions[i] == fruit_position:
                    fruit_in_body = True

        snake_positions.append(growth_position[:])

        speed = speed - 2

    # 將遊戲物件畫到螢幕上，aka 渲染
    window.blit(cover, (0, 0))
    for i in range(1, len(snake_positions)):
        window.blit(body, (project(snake_positions[i])))
    window.blit(head, project(snake_positions[0]))
    window.blit(fruit, project(fruit_position))

    font = pygame.font.SysFont(None, int(BOARD_LEN / 16))
    text = font.render("Score: " + str(len(snake_positions)), True, (255, 255, 255))
    window.blit(text, (BOARD_LEN * (3/4), 0))
    pygame.display.flip()

    # 稍等一些時間，跟遊戲進行速度有關
    time.sleep(speed / 1000)

In [ ]:
pygame.quit()